In [28]:
# sc is an existing SparkContext.
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [36]:
#Computing using RDD's

nyseData = sc.textFile("/Users/pravinkumar/Documents/Spark/testData/nyse/nyse_data")

nyseMap = nyseData.map(lambda rec: rec.split(","))

def trim(x):
    return x[3:]
nyseGroup = nyseMap.map(lambda rec: (trim(rec[1]), (rec[0], rec[6]))).groupByKey()
def calc(x, top):
    tops = []
    for d in x:
        s, p = d[0], d[1]
        tops.append(int(d[1])) 
    sorted(tops)
    topsSet = set()
    count = 0
    for i in tops:
        topsSet.add(i)
        count += 1
        if(count == top):
            break
    result = []
    for d in x:
        s, p = d[0], d[1]
        if(int(d[1]) in topsSet):
            result.append(d)
    return result   
nyseResult = nyseGroup.map(lambda rec: (rec[0], calc(rec[1], 3)))
for i in nyseResult.take(2): print(i)

('Sep-2010', [('A', '3639800'), ('AA', '24246100'), ('AAN', '554000'), ('SJM', '554000'), ('WTI', '554000')])
('Feb-2013', [('A', '2827800'), ('AA', '18242000'), ('AAN', '355600')])


In [61]:
# Coumpting using DF's

nyseRDDToDF = nyseMap.map(lambda rec: Row(date = trim(rec[1]), name = rec[0], volume = rec[6]))

nyseDF = sqlContext.createDataFrame(nyseRDDToDF)
nyseTable = nyseDF.registerTempTable("nyse")


sqlContext.sql("SELECT * \
FROM nyse as n1 \
WHERE \
( \
  n1.volume IN \
  ( \
    SELECT n2.volume \
    FROM nyse as n2 \
    ORDER BY n2.volume DESC \
    LIMIT 3 \
  ) \
)").show()




+--------+----+-------+
|    date|name| volume|
+--------+----+-------+
|Jan-2010| MOS|9999800|
|Apr-2010| CVX|9999900|
|Jun-2012| ALU|9999900|
+--------+----+-------+

